In [39]:
import numpy as np #type: ignore
import pandas as pd
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer # type: ignore
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense


In [40]:
# Load the dataset
data = pd.read_csv("spam.csv", encoding="latin-1")
messages = data["v2"]
labels = data["v1"]

In [41]:
# Preprocessing
tokenizer = Tokenizer()
tokenizer.fit_on_texts(messages)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(messages)
max_len = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')


In [42]:
# Label encoding
label_mapping = {"ham": 0, "spam": 1}
encoded_labels = np.array([label_mapping[label] for label in labels])

# Define the fNN model
model = Sequential([
    Embedding(len(word_index) + 1, 64, input_length=max_len),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

C:\Users\37655\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [43]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(padded_sequences, encoded_labels, epochs=25, batch_size=128)


Epoch 1/25
44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.8663 - loss: 0.3616
Epoch 2/25
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.9801 - loss: 0.0834
Epoch 3/25
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.9907 - loss: 0.0297
Epoch 4/25
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - accuracy: 0.9967 - loss: 0.0141
Epoch 5/25
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.9993 - loss: 0.0058
Epoch 6/25
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.9994 - loss: 0.0039
Epoch 7/25
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.9990 - loss: 0.0041
Epoch 8/25
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.9998 - loss: 0.0016
Epoch 9/25
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.9997 - loss: 0.0019
Epoch 10/25
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.9999 - loss: 8.8400e-04
Epoch 11/25
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 1.0000 - loss: 6.9218e-04
Epoch 12/25
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - a

In [44]:
# Function to predict ham or spam
def predict_message(message):
    # Preprocess the message
    sequence = tokenizer.texts_to_sequences([message])
    padded_sequence = pad_sequences(sequence, maxlen=max_len, padding='post')
    # Predict probabilities
    probabilities = model.predict(padded_sequence)
    # Convert probabilities to class labels
    prediction = "Ham" if probabilities[0] < 0.5 else "Spam"
    return prediction


In [49]:
# Predict new messages
new_messages = ["Congratulations! You've won this match get your reward from our company named ALpha Network in pakistan", "Your Collecting price is 100$ collect from this link www.asd.com.pk"]
for message in new_messages:
    predicted_label = predict_message(message)
    print(f"Message: {message}\nPredicted Label: {predicted_label}\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Message: Congratulations! You've won this match get your reward from our company named ALpha Network in pakistan
Predicted Label: Spam

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Message: Your Collecting price is 100$ collect from this link www.asd.com.pk
Predicted Label: Spam



In [46]:
# Save the tokenizer and max_len
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
with open("max_len.pkl", "wb") as f:
    pickle.dump(max_len, f)

# Save the model
model.save("spam_detection_model.h5")

In [47]:
from tensorflow import keras

loaded_model = keras.models.load_model("spam_detection_model.h5")
with open("tokenizer.pkl", "rb") as f:
    loaded_tokenizer = pickle.load(f)
with open("max_len.pkl", "rb") as f:
    loaded_max_len = pickle.load(f)